<a href="https://colab.research.google.com/github/victorviro/Deep_learning_python/blob/master/Keras_Functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Functional API

[Here](https://github.com/victorviro/Deep_learning_python/blob/master/Introduction_artificial_neural_networks_keras.ipynb) we used the Sequential API of Keras to build a MLP classifier. Although sequential models are extremely common, it is sometimes useful to build neural networks with more complex topologies, or with multiple inputs or outputs. For this purpose, Keras offers the Functional API.


One example of a non-sequential neural network is a *Wide & Deep* neural network. This neural network architecture was introduced in a [paper](https://arxiv.org/abs/1606.07792) by Heng-Tze Cheng in 2016. It connects all or part of the inputs directly to the output layer, as shown in
Figure 10-13.

![texto alternativo](https://i.ibb.co/pPrpHsz/WDNN.png)

This architecture makes it possible for the neural network to learn both
deep patterns (using the deep path) and simple rules (through the short path). In contrast, a regular MLP forces all the data to flow through the full stack of layers, thus simple patterns in the data may end up being distorted by this sequence of transformations.

Let’s build such a neural network to tackle the [California housing problem](https://scikit-learn.org/stable/datasets/index.html#california-housing-dataset):

In [1]:
# Let's load, split and scale the California housing dataset

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [2]:
# build the neural network
import keras

input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_], outputs=[output])

Using TensorFlow backend.


Let’s go through each line of this code:


- First, we need to create an `Input` object. This is needed because we may have
multiple inputs, as we will see later.

- Next, we create a `Dense` layer with 30 neurons and using the ReLU activation
function. As soon as it is created, notice that we call it like a function, passing it the input. This is why this is called the Functional API. Note that we are just telling Keras how it should connect the layers together, no actual data is being processed yet.

- We then create a second hidden layer, and again we use it as a function. Note
however that we pass it the output of the first hidden layer.

- Next, we create a `Concatenate()` layer, and once again we immediately use it like a function, to concatenate the input and the output of the second hidden layer.

- Then we create the output layer, with a single neuron and no activation function, and we call it like a function, passing it the result of the concatenation.

- Lastly, we create a Keras `Model`, specifying which inputs and outputs to use. Remember that in the Sequential API you instantiate the model object when calling `model = Sequential()` (and then add layers and constraints). In the Functional API, you create layers and then instantiate your model by calling `model = Model()` with your desired input and output layer(s) as we did.



We compile the model, train it, evaluate it and use it to make predictions.




In [3]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 30)           930         dense_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 38)           0           input_1[0][0]                    
                                                                 dense_2[0][0]              

In [4]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 0s 42us/step - loss: 2.5161 - val_loss: 9.1327
Epoch 2/20
11610/11610 [==============================] - 0s 36us/step - loss: 0.6542 - val_loss: 0.5336
Epoch 3/20
11610/11610 [==============================] - 0s 37us/step - loss: 0.5448 - val_loss: 0.4899
Epoch 4/20
11610/11610 [==============================] - 0s 36us/step - loss: 0.5159 - val_loss: 0.4697
Epoch 5/20
11610/11610 [==============================] - 0s 36us/step - loss: 0.4935 - val_loss: 0.4632
Epoch 6/20
11610/11610 [==============================] - 0s 35us/step - loss: 0.4742 - val_loss: 0.4514
Epoch 7/20
11610/11610 [==============================] - 0s 34us/step - loss: 0.4614 - val_loss: 0.4416
Epoch 8/20
11610/11610 [==============================] - 0s 35us/step - loss: 0.4501 - val_loss: 0.4484
Epoch 9/20
11610/11610 [==============================] - 0s 36us/step - loss: 0.4411 - val_loss: 0.4472
Epoch 

But what if you want to send a subset of the features through the wide path, and a different subset (possibly overlapping) through the deep path (see Figure 10-14)? 

![texto alternativo](https://i.ibb.co/KzbDTjB/multiple-inputs.png)

In this case, one solution is to use multiple inputs. For example, suppose we want to send 5 features through the deep path (features 0 to 4), and 6 features through the wide path (features 2 to 7):

In [5]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

The code is self-explanatory. Note that we specified `inputs=[input_A, input_B]`
when creating the model. Now we can compile the model as usual, but when we call
the `fit()` method, instead of passing a single input matrix `X_train` , we must pass a pair of matrices `(X_train_A, X_train_B)` : one per input. The same is true for `X_valid`, and also for `X_test` and `X_new` when you call `evaluate()` or `predict()`:

In [6]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit([X_train_A, X_train_B], y_train, epochs=20,
                    validation_data=([X_valid_A, X_valid_B], y_valid))

mse_test = model.evaluate([X_test_A, X_test_B], y_test)
y_pred = model.predict([X_new_A, X_new_B])

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 0s 41us/step - loss: 1.8974 - val_loss: 1.7702
Epoch 2/20
11610/11610 [==============================] - 0s 36us/step - loss: 0.7005 - val_loss: 0.6267
Epoch 3/20
11610/11610 [==============================] - 0s 35us/step - loss: 0.6135 - val_loss: 0.5864
Epoch 4/20
11610/11610 [==============================] - 0s 35us/step - loss: 0.5733 - val_loss: 0.5431
Epoch 5/20
11610/11610 [==============================] - 0s 36us/step - loss: 0.5451 - val_loss: 0.5195
Epoch 6/20
11610/11610 [==============================] - 0s 36us/step - loss: 0.5229 - val_loss: 0.5590
Epoch 7/20
11610/11610 [==============================] - 0s 36us/step - loss: 0.5061 - val_loss: 0.5057
Epoch 8/20
11610/11610 [==============================] - 0s 35us/step - loss: 0.4924 - val_loss: 0.4643
Epoch 9/20
11610/11610 [==============================] - 0s 36us/step - loss: 0.4796 - val_loss: 0.4563
Epoch 

There are also many use cases in which you may want to have multiple outputs:


- The task may demand it, for example you may want to locate and classify the
main object in a picture. This is both a regression task (finding the coordinates of the object’s center, as well as its width and height) and a classification task.

- Similarly, you may have multiple independent tasks to perform based on the
same data. Sure, you could train one neural network per task, but in many cases
you will get better results on all tasks by training a single neural network with one output per task. This is because the neural network can learn features in the data that are useful across tasks.

- Another use case is as a regularization technique (i.e., a training constraint whose objective is to reduce overfitting and thus improve the model’s ability to generalize). For example, you may want to add some auxiliary outputs in a neural network architecture (see Figure 10-15) to ensure that the underlying part of the network learns something useful on its own, without relying on the rest of the network.

![texto alternativo](https://i.ibb.co/pLKmRgw/multiple-outputs.png)

Adding extra outputs is quite easy: just connect them to the appropriate layers and add them to your model’s list of outputs. For example, the following code builds the network represented in Figure 10-15:

In [7]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

Each output will need its own loss function, so when we compile the model we
should pass a list of losses (if we pass a single loss, Keras will assume that the same loss must be used for all outputs). By default, Keras will compute all these losses and simply add them up to get the final loss used for training. However, we care much more about the main output than about the auxiliary output (as it is just used for regularization), so we want to give the main output’s loss a much greater weight. Fortunately, it is possible to set all the loss weights when compiling the model:

In [8]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

Now when we train the model, we need to provide some labels for each output. In
this example, the main output and the auxiliary output should try to predict the same thing, so they should use the same labels. So instead of passing `y_train` , we just need to pass `[y_train, y_train]` (and the same goes for `y_valid` and `y_test`):

In [9]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 46us/step - loss: 2.3618 - main_output_loss: 2.1316 - aux_output_loss: 4.4274 - val_loss: 2.2500 - val_main_output_loss: 1.5113 - val_aux_output_loss: 8.8912
Epoch 2/20
11610/11610 [==============================] - 0s 40us/step - loss: 1.0658 - main_output_loss: 0.8751 - aux_output_loss: 2.7818 - val_loss: 1.6173 - val_main_output_loss: 0.7666 - val_aux_output_loss: 9.2680
Epoch 3/20
11610/11610 [==============================] - 0s 39us/step - loss: 0.8144 - main_output_loss: 0.6865 - aux_output_loss: 1.9651 - val_loss: 1.4924 - val_main_output_loss: 0.6207 - val_aux_output_loss: 9.3327
Epoch 4/20
11610/11610 [==============================] - 0s 40us/step - loss: 0.7124 - main_output_loss: 0.6138 - aux_output_loss: 1.5978 - val_loss: 1.3784 - val_main_output_loss: 0.5686 - val_aux_output_loss: 8.6614
Epoch 5/20
11610/11610 [==============================] - 0s 39us/step - lo

When we evaluate the model, Keras will return the total loss, as well as all the individ‐
ual losses:

In [10]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])


5160/5160 [==============================] - 0s 18us/step


Similarly, the `predict()` method will return predictions for each output:

In [11]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

As you can see, you can build any sort of architecture you want quite easily with the
Functional API. Let’s look at one last way you can build Keras models.

## The Subclassing API

Both the Sequential API and the Functional API are declarative: you start by declaring which layers you want to use and how they should be connected, and only then can you start feeding the model some data for training or inference. This has many advantages: the model can easily be saved, cloned, shared, its structure can be displayed and analyzed, the framework can infer shapes and check types, so errors can be caught early (i.e., before any data ever goes through the model). It’s also fairly easy to debug, since the whole model is just a static graph of layers. But the flip side is just that: it’s static. Some models involve loops, varying shapes, conditional branching, and other dynamic behaviors. For such cases, or simply if you prefer a more imperative programming style, the Subclassing API is for you.

Simply subclass the `Model` class, create the layers you need in the constructor, and use them to perform the computations you want in the `call()` method. For example, creating an instance of the following `WideAndDeepModel` class gives us an equivalent model to the one we just built with the Functional API. You can then compile it, evaluate it and use it to make predictions, exactly like we just did.



In [12]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation="relu")

In [13]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=10,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 1s 45us/step - loss: 2.1914 - output_1_loss: 1.9891 - output_2_loss: 4.0119 - val_loss: 1.4701 - val_output_1_loss: 0.8327 - val_output_2_loss: 7.2024
Epoch 2/10
11610/11610 [==============================] - 0s 39us/step - loss: 0.9095 - output_1_loss: 0.7235 - output_2_loss: 2.5848 - val_loss: 1.2479 - val_output_1_loss: 0.6490 - val_output_2_loss: 6.6340
Epoch 3/10
11610/11610 [==============================] - 0s 39us/step - loss: 0.7876 - output_1_loss: 0.6537 - output_2_loss: 1.9928 - val_loss: 1.2042 - val_output_1_loss: 0.6475 - val_output_2_loss: 6.2107
Epoch 4/10
11610/11610 [==============================] - 0s 40us/step - loss: 0.7246 - output_1_loss: 0.6182 - output_2_loss: 1.6810 - val_loss: 1.1382 - val_output_1_loss: 0.6017 - val_output_2_loss: 5.9626
Epoch 5/10
11610/11610 [==============================] - 0s 40us/step - loss: 0.6804 - output_1_loss: 0.5903 - out

This example looks very much like the Functional API, except we do not need to create the inputs, we just use the input argument to the `call()` method, and we separate the creation of the layers in the constructor from their usage in the `call()` method. However, the big difference is that you can do pretty much anything you want in the `call()` method: for loops, if statements, low-level TensorFlow operations, your imagination is the limit. This makes it a great API for researchers experimenting with new ideas.

However, this extra flexibility comes at a cost: your model’s architecture is hidden within the `call()` method, so Keras cannot easily inspect it, it cannot save or clone it, and when you call the `summary()` method, you only get a list of layers, without any information on how they are connected to each other. Moreover, Keras cannot check types and shapes ahead of time, and it is easier to make mistakes. So unless you really need that extra flexibility, you should probably stick to the Sequential API or the Functional API.